Hacker news é um site criado pela startup **incubator**, onde histórias enviadas por usuários (também conhecidas como posts), obtém votos e comentários, similar ao reddit. Hacker News é popular em circulos de tecnologias e startups, a posts que estão no topo das listas podem chegar a milhares de visitantes como resultado.

O dataset pode ser encontrado [aqui](https://www.kaggle.com/hacker-news/hacker-news-posts), mas perceba que foi reduzido de 300.000 linhas para aproximadamente 20.000 removendo todos os envios que não tiveram nenhum comentário, e depois foram pegos alguns posts aleatoriamente.

Essas são as descrições das colunas:

- id: Identificador unico do Hacker News para o post
- title: O titulo do post
- url: A URL do post, se o post tiver uma
- num_points: O número de pontos ganhos, calculado pelo total de "upvotes" menos o total de "downvotes"
- num_comments: A quantidade de comentários que foi feito no post
- author: O username da pessoa que enviou o post
- created_at: Data e hora que o post foi enviado

Estamos especificamente interessados em posts do tipo **Ask HN** ou **Show HN**.
Usuários enviam **Ask HN** posts para fazer uma pergunta especifica.
Por outro lado, usuários enviam **Show HN** posts para mostrar à comunidade um projeto, produto ou compartilhar algo interessante.

Vamos comparar esses dois tipos para determinar:
- Quais deles recebem mais comentários em média?
- Posts criados em um certo horario recebem em média mais comentários?

In [1]:
# lendo o arquivo
from csv import reader
opened_file = open('hacker_news.csv', encoding='utf-8')

# transformando os dados do arquivo em uma lista
read_file = reader(opened_file)
hn = list(read_file)


In [2]:
# separar o header em uma variavel
headers = hn[0]

# o restante do dataset em outra variavel
hn = hn[1:]

In [3]:
# criando listas separadas pra cada tipo de post
ask_posts = []
show_posts = []
other_posts = []

In [4]:
for post in hn:
    title = post[1].lower()
    if title.startswith('ask hn'):
        ask_posts.append(post)
    elif title.startswith('show hn'):
        show_posts.append(post)
    else:
        other_posts.append(post)

In [5]:
print('Ask Posts:', len(ask_posts))
print('Show Posts:', len(show_posts))
print('Other Posts:', len(other_posts))

Ask Posts: 9139
Show Posts: 10158
Other Posts: 273822


Agora vou procurar saber o total de comentários pra cada tipo de post, começando pelos posts **Ask HN**

In [6]:
total_ask_comments = 0
for post in ask_posts:
    num_comments = int(post[4])
    total_ask_comments += num_comments
avg_ask_comments = total_ask_comments / len(ask_posts)

print('Total de comentários nos Posts Aks HN:', total_ask_comments)
print('Média dos comentários nos Posts Ask HN:', avg_ask_comments)

Total de comentários nos Posts Aks HN: 94986
Média dos comentários nos Posts Ask HN: 10.393478498741656


Agora fazendo para os Posts **Show HN**

In [7]:
total_show_comments = 0
for post in show_posts:
    num_comments = int(post[4])
    total_show_comments += num_comments
avg_show_comments = total_show_comments / len(show_posts)

print('Total de comentários nos Posts Aks HN:', total_show_comments)
print('Média dos comentários nos Posts Ask HN:', avg_show_comments)

Total de comentários nos Posts Aks HN: 49633
Média dos comentários nos Posts Ask HN: 4.886099625910612


A média dos Posts **Ask HN** é maior que a média dos posts **Show HN**, vamos focar o restante da análise nesse tipo de post. 
Agora vamos determinar se os Ask Posts criados em certos horários pode atrair maior número de comentários.
Vamos fazer da seguinte forma:

1. Calcular a quantidade de posts criada em cada hora do dia, e checar o número de comentários recebidos.
2. Calcular a média no número de comentários recebidos pelo horário criado

In [8]:
import datetime as dt

In [9]:
result_list = []
for post in ask_posts:
    created_at = post[6]
    num_comments = int(post[4])
    n_list = [created_at, num_comments]
    result_list.append(n_list)
    
counts_by_hour = {}
comments_by_hour = {}

for result in result_list:
    date_and_time = result[0]
    date_and_time = dt.datetime.strptime(date_and_time, '%m/%d/%Y %H:%M')
    hour = date_and_time.strftime('%H')
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = int(result[1])
    else:
        counts_by_hour[hour] = counts_by_hour[hour] + 1
        comments_by_hour[hour] = comments_by_hour[hour] + int(result[1])
        
print(counts_by_hour)
print('')
print(comments_by_hour)

{'02': 269, '01': 282, '22': 383, '21': 518, '19': 552, '17': 587, '15': 646, '14': 513, '13': 444, '11': 312, '10': 282, '09': 222, '07': 226, '03': 271, '23': 343, '20': 510, '16': 579, '08': 257, '00': 301, '18': 614, '12': 342, '04': 243, '06': 234, '05': 209}

{'02': 2996, '01': 2089, '22': 3372, '21': 4500, '19': 3954, '17': 5547, '15': 18525, '14': 4972, '13': 7245, '11': 2797, '10': 3013, '09': 1477, '07': 1585, '03': 2154, '23': 2297, '20': 4462, '16': 4466, '08': 2362, '00': 2277, '18': 4877, '12': 4234, '04': 2360, '06': 1587, '05': 1838}


In [10]:
avg_by_hour = []
for hour in comments_by_hour:
    avg_comments = comments_by_hour[hour] / counts_by_hour[hour]
    avg_by_hour.append([hour, avg_comments])
    
avg_by_hour

[['02', 11.137546468401487],
 ['01', 7.407801418439717],
 ['22', 8.804177545691905],
 ['21', 8.687258687258687],
 ['19', 7.163043478260869],
 ['17', 9.449744463373083],
 ['15', 28.676470588235293],
 ['14', 9.692007797270955],
 ['13', 16.31756756756757],
 ['11', 8.96474358974359],
 ['10', 10.684397163120567],
 ['09', 6.653153153153153],
 ['07', 7.013274336283186],
 ['03', 7.948339483394834],
 ['23', 6.696793002915452],
 ['20', 8.749019607843136],
 ['16', 7.713298791018998],
 ['08', 9.190661478599221],
 ['00', 7.5647840531561465],
 ['18', 7.94299674267101],
 ['12', 12.380116959064328],
 ['04', 9.7119341563786],
 ['06', 6.782051282051282],
 ['05', 8.794258373205741]]

A lista com o resultado da média de comentários por hora foi gerada, ficou um pouco dificil de identificar os horarios com maior média, vou ordenar os resultados e mostrar as 5 maiores médias

In [11]:
swap_avg_by_hour = []
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])

sorted_swap = sorted(swap_avg_by_hour, reverse=True)
print('Top 5 horário para comentários de Ask HN')

for row in sorted_swap[:5]:
    avg_comments = row[0]
    date_time = row[1]
    date_time = dt.datetime.strptime(date_time, '%H')
    date_time = date_time.strftime('%H:%M')
    print('{}: {:.2f} comentários em média por post'.format(date_time, avg_comments))
    
    

Top 5 horário para comentários de Ask HN
15:00: 28.68 comentários em média por post
13:00: 16.32 comentários em média por post
12:00: 12.38 comentários em média por post
02:00: 11.14 comentários em média por post
10:00: 10.68 comentários em média por post
